In [8]:
import numpy as np
import gymnasium as gym
import random
import time
from IPython.display import clear_output

In [9]:
env = gym.make('FrozenLake-v1', render_mode='ansi')

In [26]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [85]:
# define parameters
num_episodes = 9000
max_steps_per_episode = 150

learning_rate = 0.01
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001
rewards_all_episodes = []

In [86]:
print(env.reset())

(0, {'prob': 1})


In [87]:
# Q-learning algorithm
for episode in range(num_episodes):
    # initialize new episode params
    state = env.reset()[0]
    done = False
    rewards_current_episode = 0
    for step in range(max_steps_per_episode): 
        # Exploration-exploitation trade-off
        exploration_rate_threshold = random.uniform(0, 1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:]) 
        else:
            action = env.action_space.sample()
        # Take new action
        new_state, reward, done, truncated, info = env.step(action)

        # Update Q-table for (s,a)
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))

        # Set new state
        state = new_state
        # Add new reward        
        rewards_current_episode += reward
        if done:
            break
    # Exploration rate decay  
    exploration_rate = min_exploration_rate + (max_exploration_rate - exploration_decay_rate) * np.exp(-exploration_decay_rate*episode)
    # Add current episode reward to total rewards list
    rewards_all_episodes.append(rewards_current_episode)

In [88]:
# Calculate and print the average reward per thousand episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000

print("********Average reward per thousand episodes********\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

********Average reward per thousand episodes********

1000 :  0.04300000000000003
2000 :  0.18800000000000014
3000 :  0.45500000000000035
4000 :  0.5770000000000004
5000 :  0.6750000000000005
6000 :  0.7290000000000005
7000 :  0.7610000000000006
8000 :  0.7230000000000005
9000 :  0.7570000000000006


In [89]:
# Print updated Q-table
print("\n\n********Q-table********\n")
print(q_table)



********Q-table********

[[0.55622589 0.52935446 0.52841063 0.52571929]
 [0.34778043 0.32157096 0.31974628 0.50530946]
 [0.43073127 0.44100321 0.42280668 0.47591308]
 [0.27763999 0.30015299 0.30401556 0.45971898]
 [0.57301382 0.36932388 0.35708525 0.36577595]
 [0.         0.         0.         0.        ]
 [0.3585206  0.20897139 0.31019248 0.15213852]
 [0.         0.         0.         0.        ]
 [0.35304006 0.4122402  0.42878469 0.60365049]
 [0.42771544 0.65001598 0.4716544  0.37068156]
 [0.62347325 0.47458208 0.41793272 0.3236388 ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.48981288 0.51160997 0.73748837 0.50255869]
 [0.72870507 0.86641041 0.80640539 0.78139736]
 [0.         0.         0.         0.        ]]


In [91]:
# Watch our agent play Frozen Lake by playing the best action 
# from each state according to the Q-table
for episode in range(3):
    # initialize new episode params
    state = env.reset()[0]
    done = False
    print("*****EPISODE ", episode+1, "*****\n\n\n\n")
    time.sleep(1)

    for step in range(max_steps_per_episode):        
        # Show current state of environment on screen
        clear_output(wait=True)
        print(env.render())
        time.sleep(0.3)
        # Choose action with highest Q-value for current state       
        # Take new action
        action = np.argmax(q_table[state,:])        
        new_state, reward, done, truncated, info = env.step(action)
        if done:
            clear_output(wait=True)
            print(env.render())
            if reward == 1:
                print("****You reached the goal!****")
                time.sleep(3)
            else:
                print("****You fell through a hole!****")
                time.sleep(3)
                clear_output(wait=True)
            break
        # Set new state
        state = new_state

env.close()

  (Down)
SFFF
FHFH
FFFH
HFFG

****You reached the goal!****
